## Search Engine Agent With Langchain Tools

##### Wikipedia tool -- Fetches short summaries from Wikipedia

In [1]:
# Wikipedia tool
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

# Wikipedia API Wrapper with constraints
api_wrapper_wiki = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=250)

# Create the Wikipedia tool
wiki = WikipediaQueryRun(api_wrapper=api_wrapper_wiki)

##### Arxiv Tool -- Fetches research paper abstracts from Arxiv

In [2]:
# Arxiv Tool
from langchain_community.tools import ArxivQueryRun
from langchain_community.utilities import ArxivAPIWrapper

# Arxiv API Wrapper with constraints
api_wrapper_arxiv = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=250)

# Create the Arxiv tool
arxiv = ArxivQueryRun(api_wrapper=api_wrapper_arxiv)

#####  Building a Custom Tool: LangChain Docs Search (RAG)

In [4]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load LangChain documentation
loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()

# Split documents into smaller chunks
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)

# Store them in a FAISS vector database
vectordb = FAISS.from_documents(documents, OllamaEmbeddings(model="mxbai-embed-large"))

# Create a retriever tool from the vector database
retriever = vectordb.as_retriever()
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever, "langsmith-search", "Search any information about Langsmith")


##### Combining the Tools

In [5]:
tools = [wiki, arxiv, retriever_tool]

##### Setting Up the AI Brain (LLM) and Prompt

In [7]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os

load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")

# Initialize the LLM using Groq's ChatGroq with the Llama3 model
llm = ChatGroq(groq_api_key=groq_api_key, model_name="Llama3-8b-8192")

from langchain import hub
# Pull a pre-configured prompt that guides the agent's tool-calling behavior
prompt = hub.pull("hwchase17/openai-functions-agent")


/home/vpsr/Desktop/python/.venv/lib/python3.12/site-packages/langsmith/client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


##### Building the Agent

In [8]:
from langchain.agents import create_openai_tools_agent

# Create an agent with our LLM, tools, and prompt
agent = create_openai_tools_agent(llm, tools, prompt)


##### Executing the Agent


In [11]:
from langchain.agents import AgentExecutor

# Set up the agent executor, which manages the agent's decision process
agent_executor = AgentExecutor(agent=agent, tools=tools)

# Invoke the agent with different queries
response1 = agent_executor.invoke({"input": "Tell me about Langsmith"})
response2 = agent_executor.invoke({"input": "What is machine learning"})
response3 = agent_executor.invoke({"input": "What's the paper 1706.03762 about?"})

In [14]:
print(response1['output'])

It looks like the tool call yielded some information about LangSmith, a platform for building production-grade LLM (Large Language Model) applications. LangSmith seems to offer a range of features, including observability, evaluation, and prompt engineering tools to help developers build and optimize their AI applications.

Some of the key takeaways from the information provided are:

* LangSmith is a framework-agnostic platform that can be used with or without LangChain's open-source frameworks.
* The platform provides LLM-native observability, allowing developers to get meaningful insights from their applications.
* LangSmith offers evaluation tools to help developers test and optimize their applications, including automatic version control and collaboration features.
* The platform also provides prompt engineering tools to help developers find the perfect prompt for their applications.

Overall, LangSmith appears to be a powerful platform for building and optimizing AI applications,

In [15]:
print(response2['output'])

Based on the tool call id "call_25z0" and the yielded result, I will provide a more detailed and concise answer to your original question.

Machine learning is a subset of artificial intelligence that involves the use of algorithms and statistical models to enable machines to learn from data and make predictions or decisions without being explicitly programmed. It involves training models on large datasets, so they can learn patterns and relationships, and then apply this knowledge to new, unseen data.

The goal of machine learning is to enable machines to automatically improve their performance on a task without being explicitly programmed for each specific task. In other words, machines can learn from experience and adapt to new situations.

Machine learning is used in a wide range of applications, including image and speech recognition, natural language processing, recommender systems, and predictive analytics. It has many benefits, such as improved accuracy, increased efficiency, a

In [16]:
print(response3['output'])

The paper "Attention Is All You Need" by Ashish Vaswani et al. was published on August 2, 2023.
